## Explorando datos

In [1]:
# Importando librería Geopandas
import geopandas as gpd

In [2]:
# Función que nos permite 'leer' un set de datos en diferentes formatos
lista_comercios = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/data/comercios_domicilios.csv') # ruta al archivo

In [3]:
# .head() nos devuelve una parte de la tabla de datos
lista_comercios.head(10)

,id,domicilio,geometry
0,1,"11 DE SEPTIEMBRE 1240 TANDIL, PROVINCIA DE BU...",None
1,2,"14 DE JULIO 571 TANDIL, PROVINCIA DE BUENOS AI...",None
2,3,"25 DE MAYO 123 TANDIL, PROVINCIA DE BUENOS AIR...",None
3,4,"4 DE ABRIL 1471 TANDIL, PROVINCIA DE BUENOS AI...",None
4,5,"4 DE ABRIL 618 TANDIL, PROVINCIA DE BUENOS AIR...",None
5,6,"9 DE JULIO 49 TANDIL, PROVINCIA DE BUENOS AIRE...",None
6,7,"CANADA 1048 TANDIL, PROVINCIA DE BUENOS AIRES,...",None
7,8,"ALSINA 1342 TANDIL, PROVINCIA DE BUENOS AIRES,...",None
8,9,"AMEGHINO 412 TANDIL, PROVINCIA DE BUENOS AIRES...",None
9,10,"RODRIGUEZ 145 TANDIL, PROVINCIA DE BUENOS AIRE...",None


In [4]:
# Haciendo una copia del geodataframe original para luego unirla a la capa de puntos geocodificados
copia_lista_comercios = lista_comercios.copy()

In [5]:
# Georreferenciando datos
comercios_georef = gpd.tools.geocode(copia_lista_comercios['domicilio'], provider='nominatim', user_agent='demo', timeout=5)

In [6]:
# Verificando los datos del geodataframe 'comercios_georef'
comercios_georef.head()

,geometry,address
0,POINT (-59.12937 -37.31421),"1240, 11 de Septiembre, Las Ranas, Cuatro Aven..."
1,POINT (-59.14034 -37.32821),"571, 14 de Julio, Centro, Cuatro Avenidas, Tan..."
2,POINT (-59.14023 -37.33377),"123, 25 de Mayo, Centro, Cuatro Avenidas, Tand..."
3,POINT (-59.13520 -37.31317),"1471, 4 de Abril, La Estación, Cuatro Avenidas..."
4,POINT (-59.12945 -37.32326),"618, 4 de Abril, Jardín, Cuatro Avenidas, Tand..."


In [12]:
# Uniendo la capa de comercios georreferenciada con la tabla de datos originales para recuperar el campo 'id'
comercios_unida = copia_lista_comercios.join(comercios_georef, rsuffix='_')

In [13]:
# Trasladando los valores de 'geometry_' a la columna 'geometry'
comercios_unida['geometry'] = comercios_unida['geometry_']

In [14]:
# Eliminando las columnas 'geometry_' y 'address'. Definiendo un nuevo geodataframe llamado 'comercios_sin_na'
comercios_sin_na = comercios_unida.drop(columns=['geometry_','address'])

In [15]:
# Eliminando registros vacíos
comercios_sin_na['geom_empty'] = comercios_sin_na['geometry'].is_empty          # Verifico si existen fila con NA en la columna 'geometry', devuelve un valor True o False en la columna 'geom_empty'
comercios_sin_na = comercios_sin_na[comercios_sin_na['geom_empty'] == False]    # Redefino la variable 'comercios_sin_na'a partir de crear un nuevo geodataframe descartando las geometrías nulas
comercios_sin_na = comercios_sin_na.drop(columns=['geom_empty'])                # Elimino la columna auxiliar 'geom_empty'

In [ ]:
# Instalando librería mapclassify para poder utilizar .explore()
!pip install mapclassify

In [16]:
# Visualizando el resultado de la geocodificación con .explore()
comercios_sin_na.explore()

In [19]:
# Exportando el geodataframe 'comercios_sin_na' en un geojson llamado 'comercios'
comercios_sin_na.to_file('/content/drive/MyDrive/Colab Notebooks/data/comercios.geojson', driver='GeoJSON')